# Importing the required libraries

In [1]:
import pandas as pd
import numpy as np
import os
import re
import random
import pandas as pd
import functools
from nltk.tokenize import sent_tokenize
from pywsd.utils import lemmatize_sentence
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix,f1_score



Warming up PyWSD (takes ~10 secs)... took 6.484971761703491 secs.


# Taking a chunk of file and reading the chunk

In [2]:
chunk_iter = pd.read_csv("D:/QTInternship/aws_review_sofware_dataset/aws_review_sofware_dataset.csv",chunksize=10000)

#chunk_iter = pd.read_csv('your_file.csv', chunksize=1000)
df = next(chunk_iter)  # Read the first chunk

In [3]:
df.columns

Index(['Unnamed: 0', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
       'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime',
       'vote', 'image'],
      dtype='object')

In [4]:
df["words"]="default value"
df["sentences"]="default value"


for i in range(df.shape[0]):
    df.at[i,"words"]= list("")
    df.at[i,"sentences"] = list("")

In [5]:
for i in range(df.shape[0]):
    l1= sent_tokenize(str(df.loc[i,"reviewText"]))
    df.at[i,"sentences"]=l1

In [6]:
for k in range(df.shape[0]):
    df.at[k,"words"]=list("")
    for j in range(len(df.loc[k,"sentences"])):
        df.at[k,"words"].extend(lemmatize_sentence(df.loc[k,"sentences"][j]))

In [7]:
df["words_sentences"] = "default"

In [8]:
for k in range(df.shape[0]):
    df.loc[k,"words_sentences"]=functools.reduce(lambda a,b: str(a)+str(" ")+str(b), df.loc[k,"words"])

In [9]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(df.words_sentences)

# Splitting the data and Encoding the target variable


In [11]:
dense = pd.DataFrame(tfidf_matrix.todense(), columns=tfidf_vectorizer.get_feature_names_out())

In [13]:
df_y=df["verified"]

In [14]:
le=LabelEncoder()

In [15]:
df_y_1=pd.DataFrame(df_y)

In [16]:
df_y_enc=df_y_1.apply(le.fit_transform)

In [17]:
df_y_enc.head(5)

,verified
0,1
1,1
2,1
3,1
4,0


# Splitting the data into Train and Test data

In [18]:
X_Train,X_Test,Y_Train,Y_Test = train_test_split(dense,df_y_enc,test_size=0.3,random_state=45,stratify=df_y_enc)


# Model Building

In [19]:
RFC = RandomForestClassifier(random_state=45,n_estimators=1000)
RFC.fit(X_Train, Y_Train)

Y_pred_Train=RFC.predict(X_Train)
accuracy_train = accuracy_score(Y_Train, Y_pred_Train)
print(f"Accuracy of Training data is: {accuracy_train:.2f}")

print(confusion_matrix(Y_Train,Y_pred_Train))

# Generate a classification report
report = classification_report(Y_Train, Y_pred_Train)
print(f"Classification Report:\n{report}")



Y_pred_Test=RFC.predict(X_Test)
accuracy_test = accuracy_score(Y_Test, Y_pred_Test)
print(f"Accuracy of Test data is: {accuracy_test:.2f}")

print(confusion_matrix(Y_Test,Y_pred_Test))

# Generate a classification report
report = classification_report(Y_Test, Y_pred_Test)
print(f"Classification Report:\n{report}")

C:\Users\lavanya\AppData\Local\Temp\ipykernel_25844\2880767766.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RFC.fit(X_Train, Y_Train)


Accuracy of Training data is: 1.00
[[4229    3]
 [   0 2768]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4232
           1       1.00      1.00      1.00      2768

    accuracy                           1.00      7000
   macro avg       1.00      1.00      1.00      7000
weighted avg       1.00      1.00      1.00      7000

Accuracy of Test data is: 0.78
[[1626  188]
 [ 466  720]]
Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.90      0.83      1814
           1       0.79      0.61      0.69      1186

    accuracy                           0.78      3000
   macro avg       0.79      0.75      0.76      3000
weighted avg       0.78      0.78      0.78      3000



In [20]:
MNB = MultinomialNB()
MNB.fit(X_Train, Y_Train)
Y_pred_Test=MNB.predict(X_Test)
accuracy_test = accuracy_score(Y_Test, Y_pred_Test)
print(f"Accuracy of Test data is: {accuracy_test:.2f}")

print(confusion_matrix(Y_Test,Y_pred_Test))

# Generate a classification report
report = classification_report(Y_Test, Y_pred_Test)
print(f"Classification Report:\n{report}")

C:\Users\lavanya\anaconda3\envs\py39_with_tf_cpu\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of Test data is: 0.69
[[1777   37]
 [ 901  285]]
Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.98      0.79      1814
           1       0.89      0.24      0.38      1186

    accuracy                           0.69      3000
   macro avg       0.77      0.61      0.58      3000
weighted avg       0.75      0.69      0.63      3000



In [23]:
modelgbc = GradientBoostingClassifier(random_state=45,n_estimators=100)
modelgbc.fit(X_Train, Y_Train)
Y_pred_Test=modelgbc.predict(X_Test)
accuracy_test = accuracy_score(Y_Test, Y_pred_Test)
print(f"Accuracy of Test data is: {accuracy_test:.2f}")

print(confusion_matrix(Y_Test,Y_pred_Test))

# Generate a classification report
report = classification_report(Y_Test, Y_pred_Test)
print(f"Classification Report:\n{report}")

C:\Users\lavanya\anaconda3\envs\py39_with_tf_cpu\lib\site-packages\sklearn\ensemble\_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy of Test data is: 0.77
[[1592  222]
 [ 470  716]]
Classification Report:
              precision    recall  f1-score   support

           0       0.77      0.88      0.82      1814
           1       0.76      0.60      0.67      1186

    accuracy                           0.77      3000
   macro avg       0.77      0.74      0.75      3000
weighted avg       0.77      0.77      0.76      3000

